In [1]:
import numpy as np
import pandas as pd

In [2]:
choca=pd.read_excel("DBNChocapic.xls", sheet_name='Hoja3')
zuca=pd.read_excel("DBNZucaritas.xls", sheet_name='Hoja3')
zuco=pd.read_excel("DBNZucosos.xls", sheet_name='Hoja3')
resmer=pd.read_excel("DBNMercado.xls", sheet_name='Hoja3')
consum=pd.read_excel("DBNMercado.xls", sheet_name='Hoja1')

# ANOVA

In [3]:
dflist=[choca,zuca,zuco,resmer]

for d in dflist:
  d.drop(axis=0,index=range(6,d.shape[0]), inplace=True) # borra filas
  d.dropna(axis=1,inplace=True) # borra columnas

Resumen puntaje atributos por consumidores

In [4]:
nom=['C','A','O','R']

attri=pd.DataFrame(index=range(1,5),columns=nom) # se trabaja con 4 atributos (confidenciales) por cada marca
con=0
for d in dflist:
  attri[nom[con]]=d['B*C']
  con+=1
attri

,C,A,O,R
1,289.0,260.0,146.0,364.0
2,222.0,158.0,60.0,422.0
3,183.0,96.0,45.0,504.0
4,136.0,56.0,32.0,532.0


Marketing Mix Determinant Index

In [5]:
con=0
for d in dflist:
  attri[nom[con]]=attri[nom[con]]/(1772-d.loc[0,'N']) # la base para cada columna es distinta ya que no todos pensaron en todas las marcas
  con+=1
attri

,C,A,O,R
1,0.521661,0.643564,0.685446,0.244295
2,0.400722,0.391089,0.281690,0.283221
3,0.330325,0.237624,0.211268,0.338255
4,0.245487,0.138614,0.150235,0.357047


In [6]:
# Add Sum of Row and Columns
row_sum = [np.sum(attri.loc[idx]) for idx in attri.index]
attri["All"] = row_sum

col_sum = [np.sum(attri[col]) for col in attri.columns]
attri.loc["All"] = col_sum

In [7]:
attri

,C,A,O,R,All
1,0.521661,0.643564,0.685446,0.244295,2.094966
2,0.400722,0.391089,0.281690,0.283221,1.356723
3,0.330325,0.237624,0.211268,0.338255,1.117471
4,0.245487,0.138614,0.150235,0.357047,0.891383
All,1.498195,1.410891,1.328638,1.222819,5.460543


In [8]:
ctab=attri.copy()

# ANOVA Table
anova = pd.DataFrame(np.zeros((4, 4)))
anova.columns = ["SS", "DF", "MS", "F"]
anova.index = ["row", "col", "Err", "Total"]

nr=4
nc=4
# Square Sum
coef = ctab["All"]["All"]**2/(nr*nc)
ss = anova["SS"]

ss["row"] = (ctab["All"][:-1]**2).sum()/nr - coef
ss["col"] = (ctab.loc["All"][:-1]**2).sum()/nc - coef
ss["Total"] = (ctab.iloc[:-1, :-1]**2).sum().sum() - coef
ss["Err"] = ss["Total"] - ss["row"] - ss["col"]

# Degree of Freedom
df = anova["DF"]
df["row"] = nr-1
df["col"] = nc-1
df["Err"] = (nr-1)*(nc-1)
df["Tot"] = (nr*nc)-1

# Mean Square
ms = anova["MS"]
ms["row"] = ss["row"]/(nr-1)
ms["col"] = ss["col"]/(nc-1)
ms["Err"] = ss["Err"]/((nr-1)*(nc-1))

# F
anova["F"]["row"] = ms["row"]/ms["Err"]
anova["F"]["col"] = ms["col"]/ms["Err"]

In [9]:
anova

,SS,DF,MS,F
row,0.204626,3.0,0.068209,3.731613
col,0.010346,3.0,0.003449,0.188675
Err,0.164507,9.0,0.018279,0.000000
Total,0.379479,0.0,0.000000,0.000000


In [10]:
from scipy.stats import f

# P-Value - Parameters
dfn = anova.loc["row", "DF"]*anova.loc["col", "DF"]
dfd = (nr*nc)-1

# P-Value - Calculate
p_row = f.sf(anova.loc["row", "F"], dfn, dfd)
p_col = f.sf(anova.loc["col", "F"], dfn, dfd)

In [11]:
p_row, p_col

(0.01204187086990711, 0.9920568806227729)

In [12]:
# menor a 5% quiere decir que hay dependencia, por lo tanto se corrobora lo obtenido en el paper,
# la participacion de la marca depende de los 4 atributos desconocidos del marketing mix

# MARKOV

In [13]:
ind_c=consum['DESCRIP'].str.contains('CHOCAPIC')
ind_zuco=consum['DESCRIP'].str.contains('ZUCOSOS')
ind_zuca=consum['DESCRIP'].str.contains('ZUCARITAS')
consum.loc[ind_c,'DESCRIP']='C' #CHOCAPIC
consum.loc[ind_zuco,'DESCRIP']='O'  #ZUCOSOS
consum.loc[ind_zuca,'DESCRIP']='A'  #ZUCARITAS
ind_r=(consum['DESCRIP'].str.len()>1)
consum.loc[ind_r,'DESCRIP']='R' #RESTO MERCADO

In [14]:
con1=consum.groupby(by=['PANEL','CWEEK','DESCRIP']).agg(
        cantidad=pd.NamedAgg(column='DESCRIP', aggfunc='count')
).reset_index()
con1=con1[con1.duplicated(subset=['PANEL'], keep=False)] # quita todos los registros que solo tienen un historico de una semana
con1.drop_duplicates(subset=['PANEL','CWEEK'],keep='last',inplace=True) #quita los casos donde hubieron compras la misma semana y deja la con mayor frecuencia
con1['estado_i'] = con1['DESCRIP'].shift(1) # genera estado inicial
con1=con1[con1.groupby('PANEL').cumcount() != 0] # quita las primeras filas de cada grupo, por que no tienen estado inicial
con1.rename(columns={"DESCRIP": "estado_j"}, inplace=True)
con1.drop(columns=['cantidad'],inplace=True)
con1=con1[con1['CWEEK']!=1] # quita remanente de esa semana ya que no debería tener valores

In [15]:
con1

,PANEL,CWEEK,estado_j,estado_i
1,2000003,16,R,R
3,2000005,3,R,R
4,2000005,5,R,R
5,2000005,11,R,R
7,2000008,3,C,C
...,...,...,...,...
6791,55210813,18,R,R
6792,55210813,22,R,R
6793,55210813,24,R,R
6794,55210813,25,R,R


Matriz de transición

In [16]:
mat_tr=pd.crosstab(con1['estado_i'],con1['estado_j']).rename_axis(index=None, columns=None)

In [17]:
mat_tr

,A,C,O,R
A,84,35,21,172
C,37,302,18,316
O,14,21,63,116
R,178,308,113,2827


Matriz de probabilidad de transición

In [18]:
ini_val=con1['estado_i'].value_counts()
ini_val.sort_index(inplace=True)
for c in mat_tr.columns.tolist():
  mat_tr[c]=mat_tr[c]/ini_val[c]
mat_tr=mat_tr.T

In [19]:
ini_val

A     312
C     673
O     214
R    3426
Name: estado_i, dtype: int64

In [20]:
mat_tr

,A,C,O,R
A,0.269231,0.118590,0.044872,0.570513
C,0.052006,0.448737,0.031204,0.457652
O,0.098131,0.084112,0.294393,0.528037
R,0.050204,0.092236,0.033859,0.825161


In [21]:
mat_tr.sum(axis=1)

A    1.003205
C    0.989599
O    1.004673
R    1.001459
dtype: float64

Distribución a largo plazo (MS de equilibrio)

In [22]:
con1[con1['CWEEK']==2]['estado_i'].value_counts(normalize=True).sort_index().to_numpy()

array([0.09090909, 0.12121212, 0.03030303, 0.75757576])

In [23]:
MT=mat_tr.to_numpy()
MTn=MT.copy()
tol=1e-10
while (np.linalg.norm(MTn-MT.dot(MTn)) > tol):
  MTn=MT.dot(MTn)

# np.random.seed(7)
# alpha = np.random.random(4)
# alpha /= alpha.sum()
alpha=con1[con1['CWEEK']==2]['estado_i'].value_counts(normalize=True).sort_index().to_numpy() #la segunda semana porque ahí parte
Pn=alpha.dot(MTn)

print(f"Distribucion a largo plazo: {Pn}")

Distribucion a largo plazo: [0.0674874  0.14557379 0.04628944 0.74106361]


Forecast semana 3

In [24]:
con1[con1['CWEEK']==3]['estado_i'].value_counts(normalize=True).sort_index().to_numpy()

array([0.04081633, 0.14285714, 0.02040816, 0.79591837])

In [25]:
alpha.dot(MT)

array([0.07178651, 0.13759774, 0.04243304, 0.74846056])

In [26]:
Pn.dot(MT)

array([0.0674874 , 0.14557379, 0.04628944, 0.74106361])

Forecast semana 4

In [27]:
con1[con1['CWEEK']==4]['estado_i'].value_counts(normalize=True).sort_index().to_numpy()

array([0.10447761, 0.20895522, 0.01492537, 0.67164179])

In [28]:
MT2=alpha.dot(MT)
MT2.dot(MT)

array([0.06822298, 0.14286237, 0.04534862, 0.7439334 ])

In [29]:
MT2=Pn.dot(MT)
MT2.dot(MT)

array([0.0674874 , 0.14557379, 0.04628944, 0.74106361])